# CIViC Evidence Analysis
This notebook contains an analysis on CIViC evidence data

# Analysis of Normalized Queries

In [3]:
from pathlib import Path
from enum import Enum
import zipfile

import pandas as pd
from civicpy import civic as civicpy
import plotly.express as px

In [4]:
# Use latest cache that has been pushed to the repo
latest_cache_zip_path = sorted(Path().glob("cache-*.pkl.zip"))[-1]
print(f"Using {latest_cache_zip_path} for civicpy cache")

with zipfile.ZipFile(latest_cache_zip_path, "r") as zip_ref:
    zip_ref.extractall()

civicpy.load_cache(local_cache_path=Path("cache.pkl"), on_stale="ignore")

IndexError: list index out of range

## Total Variants in CIViC

In [ ]:
civic_variant_ids = civicpy.get_all_variants()
total_number_variants = len(civic_variant_ids)
f"Total Number of variants in CIViC: {total_number_variants}"

## Total Evidence items in CIViC

Need to remove all rejected evidence items

In [ ]:
civic_evidence_ids = civicpy.get_all_evidence(include_status=["accepted", "submitted"])

In [ ]:
total_ac_sub_evidence = len(civic_evidence_ids)
f"Total Number of accepted and submitted evidence items in CIViC: {total_ac_sub_evidence}"

## Total Molecular Profiles in CIViC

In [ ]:
civic_molprof_ids = civicpy.get_all_molecular_profiles(include_status=["accepted", "submitted", "rejected"])

# Create analysis functions / global variables 

In [ ]:
class VariantNormType(str, Enum):
    """Variation Normalization types"""

    NORMALIZED = "Normalized"
    UNABLE_TO_NORMALIZE = "Unable to Normalize"
    NOT_SUPPORTED = "Not Supported"


VARIANT_NORM_TYPE_VALUES = [v.value for v in VariantNormType.__members__.values()]

In [ ]:
class VariantCategory(str, Enum):
    """Create enum for the kind of variants that are in CIViC."""

    EXPRESSION = "Expression"
    EPIGENETIC_MODIFICATION = "Epigenetic Modification"
    FUSION = "Fusion"
    PROTEIN_CONS = "Protein Consequence"
    GENE_FUNC = "Gene Function"
    REARRANGEMENTS = "Rearrangements"
    COPY_NUMBER = "Copy Number"
    OTHER = "Other"
    GENOTYPES_EASY = "Genotypes Easy"
    GENOTYPES_COMPOUND = "Genotypes Compound"
    REGION_DEFINED_VAR = "Region Defined Variant"
    TRANSCRIPT_VAR = "Transcript Variant"  # no attempt to normalize these ones, since there is no query we could use


VARIANT_CATEGORY_VALUES = [v.value for v in VariantCategory.__members__.values()]

## Summary dicts 

These dictionaries will be mutated and used at the end of the analysis

In [ ]:
variant_analysis_summary = {
    "Variant Category": VARIANT_NORM_TYPE_VALUES,
    "Count of CIViC Variant Items per Category":[],
    "Fraction of all CIViC Variant Items": [],
    "Percentage of all CIViC Variant Items": [],
    "Fraction of Accepted Variant Items": [],
    "Percentage of Accepted Variant Items": [],
    "Fraction of Not Accepted Variant Items": [],
    "Percentage of Not Accepted Variant Items": []
}
variant_analysis_summary

In [ ]:
evidence_analysis_summary = {
    "Variant Category": VARIANT_NORM_TYPE_VALUES,
    "Count of CIViC Evidence Items per Category":[],
    "Fraction of all CIViC Evidence Items": [],
    "Percentage of all CIViC Evidence Items": [],
    "Fraction of Accepted Evidence Items": [],
    "Percentage of Accepted Evidence Items": [],
    "Fraction of Submitted Evidence Items": [],
    "Percentage of Submitted Evidence Items": []
}
evidence_analysis_summary

## Define Analysis Functions

In [ ]:
def variant_analysis(df: pd.DataFrame, variant_norm_type: VariantNormType) -> pd.DataFrame:
    """Do variant analysis (counts, percentages)

    :param df: Dataframe of variants
    :param variant_norm_type: The kind of variants that are in `df`
    :return: Transformed dataframe with variant ID duplicates dropped
    """
    # Drop duplicate rows
    df = df.drop_duplicates(subset=["variant_id"])
    variant_ids = list(df["variant_id"])

    # Count
    num_variants = len(variant_ids)
    fraction_variants = f"{num_variants} / {total_number_variants}"
    print(f"\nNumber of {variant_norm_type.value} Variants in CIViC: {fraction_variants}")

    # Percentage
    percentage_variants = f"{num_variants / total_number_variants * 100:.2f}%"
    print(f"Percentage of {variant_norm_type.value} Variants in CIViC: {percentage_variants}")

    # Get accepted counts
    num_accepted_variants = df.variant_accepted.sum()
    fraction_accepted_variants = f"{num_accepted_variants} / {num_variants}"
    print(f"\nNumber of accepted {variant_norm_type.value} Variants: {fraction_accepted_variants}")

    # Get accepted percentage
    percentage_accepted_variants = f"{num_accepted_variants / num_variants * 100:.2f}%"
    print(f"Percentage of accepted {variant_norm_type.value} Variants: {percentage_accepted_variants}")

    # Get not accepted counts
    num_not_accepted_variants = len(df) - num_accepted_variants
    fraction_not_accepted_variants = f"{num_not_accepted_variants} / {num_variants}"
    print(f"\nNumber of not accepted {variant_norm_type.value} Variants: {fraction_not_accepted_variants}")

    # Get not accepted percentage
    percentage_not_accepted_variants = f"{num_not_accepted_variants / num_variants * 100:.2f}%"
    print(f"Percentage of not accepted {variant_norm_type.value} Variants: {percentage_not_accepted_variants}")

    variant_analysis_summary["Count of CIViC Variant Items per Category"].append(num_variants)
    variant_analysis_summary["Fraction of all CIViC Variant Items"].append(fraction_variants)
    variant_analysis_summary["Percentage of all CIViC Variant Items"].append(percentage_variants)
    variant_analysis_summary["Fraction of Accepted Variant Items"].append(fraction_accepted_variants)
    variant_analysis_summary["Percentage of Accepted Variant Items"].append(percentage_accepted_variants)
    variant_analysis_summary["Fraction of Not Accepted Variant Items"].append(fraction_not_accepted_variants)
    variant_analysis_summary["Percentage of Not Accepted Variant Items"].append(percentage_not_accepted_variants)

    return df

In [ ]:
def transform_df_evidence_ids(df: pd.DataFrame) -> pd.DataFrame:
    """Transform dataframe to include evidence ID information

    :param df: Dataframe of variants
    :return: Transformed dataframe with evidence ID information
    """
    tmp_df = df.copy(deep=True)

    _variants_evidence_ids = []
    variant_ids = list(tmp_df["variant_id"])

    for v_id in variant_ids:
        _variant_evidence_ids = []

        for variant in civic_variant_ids:
            if int(v_id) == variant.id:
                for mp in variant.molecular_profiles:
                    for e in mp.evidence_items:
                        if e.id not in _variant_evidence_ids:
                            _variant_evidence_ids.append(e.id)

        _variants_evidence_ids.append(_variant_evidence_ids or "")

    tmp_df["evidence_ids"] = _variants_evidence_ids

    # Explode and rename evidence ids field
    tmp_df = tmp_df.explode(column="evidence_ids")
    tmp_df = tmp_df.rename(columns={"evidence_ids": "evidence_id"})

    return tmp_df

In [ ]:
def transform_df_evidence(df: pd.DataFrame) -> pd.DataFrame:
    """Transform dataframe to include evidence status, rating, and level

    :param df: Dataframe of variants
    :return: Transformed dataframe with evidence status, rating, and level information.
        Rejected evidence items will be dropped.
    """
    variants_evidence_ids = list(df["evidence_id"])

    # Add evidence status, rating, and level information
    _variants_evidence_statuses = []
    _variants_evidence_ratings = []
    _variants_evidence_levels = []

    for eid in variants_evidence_ids:
        _variant_evidence_statuses = []
        _variant_evidence_ratings = []
        _variant_evidence_levels = []

        for evidence in civic_evidence_ids:
            if eid and (int(eid) == evidence.id):
                if evidence.status not in _variant_evidence_statuses:
                    _variant_evidence_statuses.append(evidence.status)

                if evidence.rating not in _variant_evidence_ratings:
                    _variant_evidence_ratings.append(evidence.rating)

                if evidence.evidence_level not in _variant_evidence_levels:
                    _variant_evidence_levels.append(evidence.evidence_level)
                    
        _variants_evidence_statuses.append(_variant_evidence_statuses or "")
        _variants_evidence_ratings.append(_variant_evidence_ratings or "")
        _variants_evidence_levels.append(_variant_evidence_levels or "")

    df["evidence_status"] = _variants_evidence_statuses
    df["evidence_status"] = df["evidence_status"].str.join(", ")
    df["evidence_rating"] = _variants_evidence_ratings
    df["evidence_level"] = _variants_evidence_levels

    # Drop rejected evidence items
    df = df.drop(df[df.evidence_status == "rejected"].index)

    return df


In [ ]:
def evidence_analysis(df: pd.DataFrame, variant_norm_type: VariantNormType) -> pd.DataFrame:
    """Do evidence analysis (counts, percentages)

    :param df: Dataframe of variants
    :param variant_norm_type: The kind of variants that are in `df`
    :return: Transformed dataframe with evidence ID duplicates dropped
    """
    # Count
    num_variant_unique_evidence_items = len(set(df.evidence_id))
    fraction_evidence_items = f"{num_variant_unique_evidence_items} / {total_ac_sub_evidence}"
    print(f"Number of {variant_norm_type.value} Variant Evidence items in CIViC: {fraction_evidence_items}")

    # Percentage
    percentage_evidence_items = f"{num_variant_unique_evidence_items / total_ac_sub_evidence * 100:.2f}%"
    print(f"Percentage of {variant_norm_type.value} Variant Evidence items in CIViC: {percentage_evidence_items}")

    # Add evidence accepted column
    df["evidence_accepted"] = df.evidence_status.map({"accepted": True, "submitted": False})

    # Drop evidence id duplicates
    df = df.drop_duplicates(subset=["evidence_id"])

    # Get accepted counts
    num_accepted_evidences_variants = df.evidence_accepted.sum()
    fraction_accepted_evidences_variants = f"{num_accepted_evidences_variants} / {num_variant_unique_evidence_items}"
    print(f"\nNumber of accepted {variant_norm_type.value} Variant Evidence items: {fraction_accepted_evidences_variants}")

    # Get accepted percentage
    percentage_accepted_evidences_variants = f"{num_accepted_evidences_variants / num_variant_unique_evidence_items * 100:.2f}%"
    print(f"Percentage of accepted {variant_norm_type.value} Variant Evidence items: {percentage_accepted_evidences_variants}")

    # Get submitted counts
    number_submitted_evidences_variants = len(df) - num_accepted_evidences_variants
    fraction_submitted_evidences_variants = f"{number_submitted_evidences_variants} / {num_variant_unique_evidence_items}"
    print(f"\nNumber of submitted {variant_norm_type.value} Variant Evidence items: {fraction_submitted_evidences_variants}")

    # Get submitted percentage
    percentage_submitted_evidences_variants = f"{number_submitted_evidences_variants / num_variant_unique_evidence_items * 100:.2f}%"
    print(f"Percentage of not submitted {variant_norm_type.value} Variant Evidence items: {percentage_submitted_evidences_variants}")

    evidence_analysis_summary["Count of CIViC Evidence Items per Category"].append(num_variant_unique_evidence_items)
    evidence_analysis_summary["Fraction of all CIViC Evidence Items"].append(fraction_evidence_items)
    evidence_analysis_summary["Percentage of all CIViC Evidence Items"].append(percentage_evidence_items)
    evidence_analysis_summary["Fraction of Accepted Evidence Items"].append(fraction_accepted_evidences_variants)
    evidence_analysis_summary["Percentage of Accepted Evidence Items"].append(percentage_accepted_evidences_variants)
    evidence_analysis_summary["Fraction of Submitted Evidence Items"].append(fraction_submitted_evidences_variants)
    evidence_analysis_summary["Percentage of Submitted Evidence Items"].append(percentage_submitted_evidences_variants)

    return df

In [ ]:
def transform_df_mp_id(df: pd.DataFrame) -> pd.DataFrame:
    """Transform dataframe to include molecular profile ID information

    :param df: Dataframe of variants
    :return: Transformed dataframe with molecular profile ID information
    """
    tmp_df = df.copy(deep=True)

    variants_molprof_ids = []
    variant_ids = list(tmp_df["variant_id"])

    for v_id in variant_ids:
        variant_molprof_ids = []

        for variant in civic_variant_ids:
            if int(v_id) == variant.id:
                for mp in variant.molecular_profiles:
                    if mp.id not in variant_molprof_ids:
                        variant_molprof_ids.append(mp.id)

        variants_molprof_ids.append(variant_molprof_ids or "")

    tmp_df["molecular_profile_id"] = variants_molprof_ids
    return tmp_df

In [ ]:
def transform_df_mp_score(df: pd.DataFrame) -> pd.DataFrame:
    """Transform dataframe to include molecular profile score information

    :param df: Dataframe of variants
    :return: Transformed dataframe with molecular profile score information
    """
    variants_molprof_scores = []
    normalized_variant_molprof_ids = list(df["molecular_profile_id"])

    for mp_ids in normalized_variant_molprof_ids:
        variant_molprof_scores = []
        for mp_id in mp_ids:
            for molprof in civic_molprof_ids:
                if int(mp_id) == molprof.id:
                    if molprof.molecular_profile_score not in variant_molprof_scores:
                        variant_molprof_scores.append(molprof.molecular_profile_score)

        variants_molprof_scores.append(variant_molprof_scores or "")

    df["molecular_profile_score"] = variants_molprof_scores
    return df

In [ ]:
def transform_df_mp_score_sum(df: pd.DataFrame) -> pd.DataFrame:
    """Transform dataframe to include molecular profile score sum information

    :param df: Dataframe of variants
    :return: Transformed dataframe with molecular profile score sum information
    """
    df["molecular_profile_score_sum"] = df["molecular_profile_score"].apply(lambda x: sum(x))
    return df

## List of Normalized Variants ID's

In [ ]:
normalized_queries_df = pd.read_csv("able_to_normalize_queries.csv", sep="\t")
normalized_queries_df.head()

## Variant analysis

In [ ]:
normalized_queries_df = variant_analysis(normalized_queries_df, VariantNormType.NORMALIZED)
normalized_queries_df.head()

In [ ]:
variant_analysis_summary

## Transform df for evidence analysis

In [ ]:
normalized_queries_add_evidence_df = transform_df_evidence_ids(normalized_queries_df)
normalized_queries_add_evidence_df.head()

In [ ]:
normalized_queries_add_evidence_df = transform_df_evidence(normalized_queries_add_evidence_df)
normalized_queries_add_evidence_df.head()

## Evidence analysis

In [ ]:
normalized_queries_add_evidence_df = evidence_analysis(normalized_queries_add_evidence_df, VariantNormType.NORMALIZED)
normalized_queries_add_evidence_df.head()

## Impact
molecular profile score

### Import molecular profile id

In [ ]:
normalized_queries_add_molprof_df = transform_df_mp_id(normalized_queries_df)
normalized_queries_add_molprof_df.head()

In [ ]:
normalized_queries_add_molprof_df.loc[normalized_queries_add_molprof_df["variant_id"] == 190]

### Import molecular profile scores

In [ ]:
normalized_queries_add_molprof_df = transform_df_mp_score(normalized_queries_add_molprof_df)
normalized_queries_add_molprof_df.head()

In [ ]:
normalized_queries_add_molprof_df.loc[normalized_queries_add_molprof_df["variant_id"] == 190]

In [ ]:
normalized_queries_add_molprof_df = transform_df_mp_score_sum(normalized_queries_add_molprof_df)
normalized_queries_add_molprof_df.head()

In [ ]:
normalized_queries_add_molprof_df.loc[normalized_queries_add_molprof_df["variant_id"] == 190]

# Analysis of Unable to Normalize Queries

## List of Unable to Normalize Variant ID's

In [ ]:
not_normalized_queries_df = pd.read_csv("unable_to_normalize_queries.csv", sep="\t")
not_normalized_queries_df.head()

## Variant analysis

In [ ]:
not_normalized_queries_df = variant_analysis(not_normalized_queries_df, VariantNormType.UNABLE_TO_NORMALIZE)
not_normalized_queries_df.head()

## Transform df for evidence analysis

In [ ]:
not_normalized_quer_add_evidence_df = transform_df_evidence_ids(not_normalized_queries_df)
not_normalized_quer_add_evidence_df.head()

In [ ]:
not_normalized_quer_add_evidence_df = transform_df_evidence(not_normalized_quer_add_evidence_df)
not_normalized_quer_add_evidence_df.head()

## Evidence analysis

In [ ]:
not_normalized_quer_add_evidence_df = evidence_analysis(not_normalized_quer_add_evidence_df, VariantNormType.UNABLE_TO_NORMALIZE)
not_normalized_quer_add_evidence_df.head()

## Impact
molecular profile score

### Import molecular profile id

In [ ]:
not_normalized_queries_add_molprof_df = transform_df_mp_id(not_normalized_queries_df)
not_normalized_queries_add_molprof_df.head()

### Import molecular profile scores

In [ ]:
not_normalized_queries_add_molprof_df = transform_df_mp_score(not_normalized_queries_add_molprof_df)
not_normalized_queries_add_molprof_df.head()

In [ ]:
not_normalized_queries_add_molprof_df = transform_df_mp_score_sum(not_normalized_queries_add_molprof_df)
not_normalized_queries_add_molprof_df.head()

# Analysis of Not Supported Variants

### List of Not Supported Variant ID's

In [ ]:
not_supported_queries_df = pd.read_csv("not_supported_variants.csv", sep="\t")
not_supported_queries_df.head()

## Variant Analysis

In [ ]:
not_supported_queries_df = variant_analysis(not_supported_queries_df, VariantNormType.NOT_SUPPORTED)
not_supported_queries_df.head()

### Not Supported Variant Analysis by Subcategory 

In [ ]:
not_supported_variant_analysis_summary = {
    "Category": VARIANT_CATEGORY_VALUES,
    "Count of CIViC Variant Items per Category":[],
    "Fraction of Not Supported Variant Items": [],
    "Percent of Not Supported Variant Items": [],
    "Fraction of all CIViC Variant Items": [],
    "Percent of all CIViC Variant Items": [],
    "Fraction of Accepted Variant Items": [],
    "Percent of Accepted Variant Items": [],
    "Fraction of Not Accepted Variant Items": [],
    "Percent of Not Accepted Variant Items": []
}

In [ ]:
not_supported_variant_categories_summary_data = dict()
total_number_unique_not_supported_variants = len(set(not_supported_queries_df.variant_id))

for category in VARIANT_CATEGORY_VALUES:  # These are not supported categories
    not_supported_variant_categories_summary_data[category] = {}
    category_df = not_supported_queries_df[not_supported_queries_df.category == category]

    # Count
    number_unique_not_supported_category_variants = len(set(category_df.variant_id))
    not_supported_variant_categories_summary_data[category]["number_unique_not_supported_category_variants"] = number_unique_not_supported_category_variants

    # Fraction
    fraction_not_supported_category_variant_of_civic = f"{number_unique_not_supported_category_variants} / {total_number_variants}"
    not_supported_variant_categories_summary_data[category]["fraction_not_supported_category_variant_of_civic"] = fraction_not_supported_category_variant_of_civic

    # Percent
    percent_not_supported_category_variant_of_civic = f"{number_unique_not_supported_category_variants / total_number_variants * 100:.2f}%"
    not_supported_variant_categories_summary_data[category]["percent_not_supported_category_variant_of_civic"] = percent_not_supported_category_variant_of_civic

    # Not supported fraction
    fraction_not_supported_category_variant_of_total_not_supported = f"{number_unique_not_supported_category_variants} / {total_number_unique_not_supported_variants}"
    not_supported_variant_categories_summary_data[category]["fraction_not_supported_category_variant_of_total_not_supported"] = fraction_not_supported_category_variant_of_total_not_supported

    # Not supported percent
    percent_not_supported_category_variant_of_total_not_supported = f"{number_unique_not_supported_category_variants / total_number_unique_not_supported_variants * 100:.2f}%"
    not_supported_variant_categories_summary_data[category]["percent_not_supported_category_variant_of_total_not_supported"] = percent_not_supported_category_variant_of_total_not_supported

    # Accepted fraction
    number_accepted_not_supported_category_variants = category_df.variant_accepted.sum()
    fraction_accepted_not_supported_category_variants = f"{number_accepted_not_supported_category_variants} / {number_unique_not_supported_category_variants}"
    not_supported_variant_categories_summary_data[category]["fraction_accepted_not_supported_category_variants"] = fraction_accepted_not_supported_category_variants

    # Accepted percent
    percentage_accepted_not_supported_category_variants = f"{number_accepted_not_supported_category_variants / number_unique_not_supported_category_variants * 100:.2f}%"
    not_supported_variant_categories_summary_data[category]["percentage_accepted_not_supported_category_variants"] = percentage_accepted_not_supported_category_variants

    # Not accepted fraction
    number_not_accepted_not_supported_category_variants = len(category_df) - number_accepted_not_supported_category_variants
    fraction_not_accepted_not_supported_category_variants = f" {number_not_accepted_not_supported_category_variants} / {number_unique_not_supported_category_variants}"
    not_supported_variant_categories_summary_data[category]["fraction_not_accepted_not_supported_category_variants"] = fraction_not_accepted_not_supported_category_variants

    # Not accepted percent
    percentage_not_accepted_not_supported_category_variants = f"{number_not_accepted_not_supported_category_variants / number_unique_not_supported_category_variants * 100:.2f}%"
    not_supported_variant_categories_summary_data[category]["percentage_not_accepted_not_supported_category_variants"] = percentage_not_accepted_not_supported_category_variants

    not_supported_variant_analysis_summary["Count of CIViC Variant Items per Category"].append(number_unique_not_supported_category_variants)
    not_supported_variant_analysis_summary["Fraction of all CIViC Variant Items"].append(fraction_not_supported_category_variant_of_civic)
    not_supported_variant_analysis_summary["Percent of all CIViC Variant Items"].append(percent_not_supported_category_variant_of_civic)
    not_supported_variant_analysis_summary["Fraction of Not Supported Variant Items"].append(fraction_not_supported_category_variant_of_total_not_supported)
    not_supported_variant_analysis_summary["Percent of Not Supported Variant Items"].append(percent_not_supported_category_variant_of_total_not_supported)
    not_supported_variant_analysis_summary["Fraction of Accepted Variant Items"].append(fraction_accepted_not_supported_category_variants)
    not_supported_variant_analysis_summary["Percent of Accepted Variant Items"].append(percentage_accepted_not_supported_category_variants)
    not_supported_variant_analysis_summary["Fraction of Not Accepted Variant Items"].append(fraction_not_accepted_not_supported_category_variants)
    not_supported_variant_analysis_summary["Percent of Not Accepted Variant Items"].append(percentage_not_accepted_not_supported_category_variants)

## Transform df for evidence analysis

In [ ]:
not_supported_variants_add_evidence_df = transform_df_evidence_ids(not_supported_queries_df)
not_supported_variants_add_evidence_df.head()

In [ ]:
not_supported_variants_add_evidence_df.loc[not_supported_variants_add_evidence_df['evidence_id'] == '']

In [ ]:
not_supported_variants_add_evidence_df = transform_df_evidence(not_supported_variants_add_evidence_df)
not_supported_variants_add_evidence_df.head()

## Evidence analysis

In [ ]:
not_supported_variants_add_evidence_df = evidence_analysis(not_supported_variants_add_evidence_df, VariantNormType.NOT_SUPPORTED)
not_supported_variants_add_evidence_df.head()

### Not Supported Variant Evidence Analysis by Subcategory 

 List all the possible variant categories, have to use non unique file since evidence items are used more than once across groups


In [ ]:
not_supported_variant_categories = not_supported_variants_add_evidence_df.category.unique()
[v for v in not_supported_variant_categories]

Evidence ID's can qualify for more than one kind of variant category

In [ ]:
duplicate = not_supported_variants_add_evidence_df[not_supported_variants_add_evidence_df.duplicated("evidence_id", keep=False)]
duplicate

In [ ]:
not_supported_variant_evidence_summary = {
    "Category": VARIANT_CATEGORY_VALUES,
    "Count of CIViC Evidence Items per Category":[],
    "Fraction of CIViC Evidence Items": [],
    "Percent of all CIViC Evidence Items": [],
    "Fraction of Not Supported Variant Evidence Items": [],
    "Percent of Not Supported Variant Evidence Items": [],
    "Fraction of Accepted Evidence Items": [],
    "Percent of Accepted Evidence Items": [],
    "Fraction of Submitted Evidence Items": [],
    "Percent of Submitted Evidence Items": []
}

In [ ]:
not_supported_variant_categories_evidence_summary_data = dict()
total_number_not_supported_variant_unique_evidence_items = len(set(not_supported_variants_add_evidence_df.evidence_id))

for category in VARIANT_CATEGORY_VALUES:
    not_supported_variant_categories_evidence_summary_data[category] = {}
    evidence_category_df = not_supported_variants_add_evidence_df[not_supported_variants_add_evidence_df.category == category]
    evidence_category_df = evidence_category_df.drop_duplicates(subset=["evidence_id"])

    # Count
    number_unique_not_supported_category_evidence = len(set(evidence_category_df.evidence_id))
    not_supported_variant_categories_evidence_summary_data[category]["number_unique_not_supported_category_evidence"] = number_unique_not_supported_category_evidence

    # Fraction
    fraction_not_supported_category_variant_evidence_of_civic = f"{number_unique_not_supported_category_evidence} / {total_ac_sub_evidence}"
    not_supported_variant_categories_evidence_summary_data[category]["fraction_not_supported_category_variant_evidence_of_civic"] = fraction_not_supported_category_variant_evidence_of_civic

    # Percent
    percent_not_supported_category_variant_evidence_of_civic = f"{number_unique_not_supported_category_evidence / total_ac_sub_evidence * 100:.2f}%"
    not_supported_variant_categories_evidence_summary_data[category]["percent_not_supported_category_variant_evidence_of_civic"] = percent_not_supported_category_variant_evidence_of_civic

    # Not supported fraction
    fraction_not_supported_category_variant_evidence_of_total_not_supported = f"{number_unique_not_supported_category_evidence} / {total_number_not_supported_variant_unique_evidence_items}"
    not_supported_variant_categories_evidence_summary_data[category]["fraction_not_supported_category_variant_evidence_of_total_not_supported"] = fraction_not_supported_category_variant_evidence_of_total_not_supported

    # Not supported percent
    percent_not_supported_category_variant_evidence_of_total_not_supported = f"{number_unique_not_supported_category_evidence / total_number_not_supported_variant_unique_evidence_items * 100:.2f}%"
    not_supported_variant_categories_evidence_summary_data[category]["percent_not_supported_category_variant_evidence_of_total_not_supported"] = percent_not_supported_category_variant_evidence_of_total_not_supported

    # Accepted fraction
    number_accepted_not_supported_category_variant_evidence = evidence_category_df.evidence_accepted.sum()
    fraction_accepted_evidence_not_supported_category_variants = f"{number_accepted_not_supported_category_variant_evidence} / {number_unique_not_supported_category_evidence}"
    not_supported_variant_categories_evidence_summary_data[category]["fraction_accepted_evidence_not_supported_category_variants"] = fraction_accepted_evidence_not_supported_category_variants

    # Accepted percent
    percentage_accepted_evidence_not_supported_category_variants = f"{number_accepted_not_supported_category_variant_evidence / number_unique_not_supported_category_evidence * 100:.2f}%"
    not_supported_variant_categories_evidence_summary_data[category]["percentage_accepted_evidence_not_supported_category_variants"] = percentage_accepted_evidence_not_supported_category_variants

    # Submitted fraction
    number_submitted_not_supported_category_variant_evidence = number_unique_not_supported_category_evidence - evidence_category_df.evidence_accepted.sum()
    fraction_submitted_evidence_not_supported_category_variants = f"{number_submitted_not_supported_category_variant_evidence} / {number_unique_not_supported_category_evidence}"
    not_supported_variant_categories_evidence_summary_data[category]["fraction_submitted_evidence_not_supported_category_variants"] = fraction_submitted_evidence_not_supported_category_variants

    # Submitted percent
    percentage_submitted_evidence_not_supported_category_variants = f"{number_submitted_not_supported_category_variant_evidence / number_unique_not_supported_category_evidence * 100:.2f}%"
    not_supported_variant_categories_evidence_summary_data[category]["percentage_submitted_evidence_not_supported_category_variants"] = percentage_submitted_evidence_not_supported_category_variants
   
    not_supported_variant_evidence_summary["Count of CIViC Evidence Items per Category"].append(number_unique_not_supported_category_evidence)
    not_supported_variant_evidence_summary["Fraction of CIViC Evidence Items"].append(fraction_not_supported_category_variant_evidence_of_civic)
    not_supported_variant_evidence_summary["Percent of all CIViC Evidence Items"].append(percent_not_supported_category_variant_evidence_of_civic)
    not_supported_variant_evidence_summary["Fraction of Not Supported Variant Evidence Items"].append(fraction_not_supported_category_variant_evidence_of_total_not_supported)
    not_supported_variant_evidence_summary["Percent of Not Supported Variant Evidence Items"].append(percent_not_supported_category_variant_evidence_of_total_not_supported)
    not_supported_variant_evidence_summary["Fraction of Accepted Evidence Items"].append(fraction_accepted_evidence_not_supported_category_variants)
    not_supported_variant_evidence_summary["Percent of Accepted Evidence Items"].append(percentage_accepted_evidence_not_supported_category_variants)
    not_supported_variant_evidence_summary["Fraction of Submitted Evidence Items"].append(fraction_submitted_evidence_not_supported_category_variants)
    not_supported_variant_evidence_summary["Percent of Submitted Evidence Items"].append(percentage_submitted_evidence_not_supported_category_variants)

## Impact
molecular profile score

### Import molecular profile id

In [ ]:
not_supported_variants_add_molprof_df = transform_df_mp_id(not_supported_queries_df)
not_supported_variants_add_molprof_df.head()

### Import molecular profile scores

In [ ]:
not_supported_variants_add_molprof_df = transform_df_mp_score(not_supported_variants_add_molprof_df)
not_supported_variants_add_molprof_df.head()

In [ ]:
not_supported_variants_add_molprof_df = transform_df_mp_score_sum(not_supported_variants_add_molprof_df)
not_supported_variants_add_molprof_df.head()

### Impact by Subcategory

In [ ]:
not_supported_impact_summary = {
    "Category": VARIANT_CATEGORY_VALUES,
    "CIVIC Total Sum Impact Score": [],
    'Average Impact Score per Variant': [],
    'Average Impact Score per Evidence Item':[],
    "Total Number Evidence Items": [v["number_unique_not_supported_category_evidence"] for v in not_supported_variant_categories_evidence_summary_data.values()],
    "% Accepted Evidence Items": [v["percentage_accepted_evidence_not_supported_category_variants"] for v in not_supported_variant_categories_evidence_summary_data.values()],
    "Total Number Variants": [v["number_unique_not_supported_category_variants"] for v in not_supported_variant_categories_summary_data.values()]
}

In [ ]:
not_supported_variant_categories_impact_data = dict()
for category in VARIANT_CATEGORY_VALUES:
    not_supported_variant_categories_impact_data[category] = {}
    impact_category_df = not_supported_variants_add_molprof_df[not_supported_variants_add_molprof_df.category == category]

    total_sum_not_supported_category_impact = impact_category_df["molecular_profile_score_sum"].sum()
    not_supported_variant_categories_impact_data[category]["total_sum_not_supported_category_impact"] = total_sum_not_supported_category_impact

    avg_impact_score_variant = total_sum_not_supported_category_impact/ number_unique_not_supported_category_variants
    not_supported_variant_categories_impact_data[category]["avg_impact_score_variant"]=  avg_impact_score_variant

    avg_impact_score_evidence = total_sum_not_supported_category_impact/ number_unique_not_supported_category_evidence
    not_supported_variant_categories_impact_data[category]["avg_impact_score_evidence"]=  avg_impact_score_evidence

    not_supported_impact_summary["CIVIC Total Sum Impact Score"].append(total_sum_not_supported_category_impact)
    not_supported_impact_summary["Average Impact Score per Variant"].append(avg_impact_score_variant)
    not_supported_impact_summary["Average Impact Score per Evidence Item"].append(avg_impact_score_evidence)


    print(number_unique_not_supported_category_variants)
    print(f"{category}: {total_sum_not_supported_category_impact}")

In [ ]:
not_supported_variant_impact_df = pd.DataFrame(not_supported_impact_summary)

In [ ]:
not_supported_variant_impact_df

In [ ]:
not_supported_variant_impact_df.to_csv('../not_supported_variant_impact_df.csv', index=False)

# Summary

## Variant Analysis

In [ ]:
all_variant_df = pd.DataFrame(variant_analysis_summary)

In [ ]:
all_variant_df["Percentage of all CIViC Variant Items"] = all_variant_df["Fraction of all CIViC Variant Items"].astype(str) + "  (" + all_variant_df["Percentage of all CIViC Variant Items"] + ")"
all_variant_df["Percentage of Accepted Variant Items"] = all_variant_df["Fraction of Accepted Variant Items"].astype(str) + "  (" + all_variant_df["Percentage of Accepted Variant Items"] + ")"
all_variant_df["Percentage of Not Accepted Variant Items"] = all_variant_df["Fraction of Not Accepted Variant Items"].astype(str) + "  (" + all_variant_df["Percentage of Not Accepted Variant Items"] + ")"

In [ ]:
all_variant_df = all_variant_df.drop(["Fraction of all CIViC Variant Items", "Fraction of Accepted Variant Items", "Fraction of Not Accepted Variant Items"], axis=1)

In [ ]:
all_variant_percent_status_df = all_variant_df.drop(["Percentage of all CIViC Variant Items", "Count of CIViC Variant Items per Category"], axis=1)

for_merge_all_variant_percent_of_civic_df = all_variant_df.drop(["Percentage of Accepted Variant Items", "Percentage of Not Accepted Variant Items"], axis=1)

all_variant_percent_of_civic_df = for_merge_all_variant_percent_of_civic_df.drop(["Count of CIViC Variant Items per Category"], axis=1)

In [ ]:
for_merge_all_variant_percent_of_civic_df.to_csv('../for_merge_all_variant_percent_of_civic_df.csv', index=False)


Summary Table 1: The table below shows the 3 categories that CIViC variants were divided into after normalization and what percentage they make up of all variants in CIViC data. 

In [ ]:
all_variant_percent_of_civic_df = all_variant_percent_of_civic_df.set_index("Variant Category")
all_variant_percent_of_civic_df

In [ ]:
civic_summary_table_1 = all_variant_percent_of_civic_df

Summary Table 2: The table below shows the 3 categories that CIViC variants were divided into after normalization and what percentage of the variants in each category are accepted (have at least one evidence item that is accepted) or not.

In [ ]:
all_variant_percent_status_df = all_variant_percent_status_df.set_index("Variant Category")
all_variant_percent_status_df

In [ ]:
civic_summary_table_2 = all_variant_percent_status_df

In [ ]:
not_supported_variant_df = pd.DataFrame(not_supported_variant_analysis_summary)

In [ ]:
not_supported_variant_df["Percent of Not Supported Variant Items"] = not_supported_variant_df["Fraction of Not Supported Variant Items"].astype(str) + "  (" + not_supported_variant_df["Percent of Not Supported Variant Items"] + ")"
not_supported_variant_df["Percent of all CIViC Variant Items"] = not_supported_variant_df["Fraction of all CIViC Variant Items"].astype(str) + "  (" + not_supported_variant_df["Percent of all CIViC Variant Items"] + ")"
not_supported_variant_df["Percent of Accepted Variant Items"] = not_supported_variant_df["Fraction of Accepted Variant Items"].astype(str) + "  (" + not_supported_variant_df["Percent of Accepted Variant Items"] + ")"
not_supported_variant_df["Percent of Not Accepted Variant Items"] = not_supported_variant_df["Fraction of Not Accepted Variant Items"].astype(str) + "  (" + not_supported_variant_df["Percent of Not Accepted Variant Items"] + ")"

In [ ]:
not_supported_variant_df = not_supported_variant_df.drop(["Fraction of Not Supported Variant Items", "Fraction of all CIViC Variant Items", "Fraction of Accepted Variant Items", "Fraction of Not Accepted Variant Items"], axis=1)

In [ ]:
for_merge_not_supported_variant_percent_of_civic_df = not_supported_variant_df.drop(["Percent of Not Supported Variant Items", "Percent of Accepted Variant Items", "Percent of Not Accepted Variant Items"], axis=1)

not_supported_variant_percent_of_civic_df = for_merge_not_supported_variant_percent_of_civic_df.drop(["Count of CIViC Variant Items per Category"], axis=1)

not_supported_variant_percent_of_not_supported_df = not_supported_variant_df.drop(["Percent of all CIViC Variant Items", "Count of CIViC Variant Items per Category", "Percent of Accepted Variant Items", "Percent of Not Accepted Variant Items"], axis=1)

not_supported_variant_percent_evidence_df = not_supported_variant_df.drop(["Percent of all CIViC Variant Items", "Percent of Not Supported Variant Items","Count of CIViC Variant Items per Category"], axis=1)

In [ ]:
for_merge_not_supported_variant_percent_of_civic_df.to_csv('../for_merge_not_supported_variant_percent_of_civic_df.csv', index=False)

Summary Table 3: The table below shows the categories that the Not Supported variants were broken into and what percentage of all CIViC variants they make up. These percentages will not add up to 100% because Not Supported variants make up 45.62% of all CIViC variants. 

In [ ]:
not_supported_variant_percent_of_civic_df = not_supported_variant_percent_of_civic_df.set_index("Category")
not_supported_variant_percent_of_civic_df

In [ ]:
civic_summary_table_3 = not_supported_variant_percent_of_civic_df

Summary Table 4: The table below shows the Not Supported variants broken up into 12 sub categories and what percent each sub category take up in Not Supported variant group.

In [ ]:
not_supported_variant_percent_of_not_supported_df = not_supported_variant_percent_of_not_supported_df.set_index("Category")
not_supported_variant_percent_of_not_supported_df

In [ ]:
civic_summary_table_4= not_supported_variant_percent_of_not_supported_df

Summary Table 5: The table below shows the percent of variant items in each Not Supported variant sub category that are accepted(have at least one evidence item that is accepted) or not, per category.

In [ ]:
not_supported_variant_percent_evidence_df = not_supported_variant_percent_evidence_df.set_index("Category")
not_supported_variant_percent_evidence_df

In [ ]:
civic_summary_table_5 = not_supported_variant_percent_evidence_df

## Evidence Analysis

In [ ]:
all_variant_evidence_df = pd.DataFrame(evidence_analysis_summary)

In [ ]:
all_variant_evidence_df["Percentage of all CIViC Evidence Items"] = all_variant_evidence_df["Fraction of all CIViC Evidence Items"].astype(str) + "  (" + all_variant_evidence_df["Percentage of all CIViC Evidence Items"] + ")"
all_variant_evidence_df["Percentage of Accepted Evidence Items"] = all_variant_evidence_df["Fraction of Accepted Evidence Items"].astype(str) + "  (" + all_variant_evidence_df["Percentage of Accepted Evidence Items"] + ")"
all_variant_evidence_df["Percentage of Submitted Evidence Items"] = all_variant_evidence_df["Fraction of Submitted Evidence Items"].astype(str) + "  (" + all_variant_evidence_df["Percentage of Submitted Evidence Items"] + ")"

In [ ]:
all_variant_evidence_df = all_variant_evidence_df.drop(["Fraction of all CIViC Evidence Items", "Fraction of Accepted Evidence Items", "Fraction of Submitted Evidence Items"], axis=1)

In [ ]:
for_merge_all_variant_evidence_percent_of_civic_df = all_variant_evidence_df.drop(["Percentage of Accepted Evidence Items", "Percentage of Submitted Evidence Items"], axis=1)

all_variant_evidence_percent_of_civic_df = for_merge_all_variant_evidence_percent_of_civic_df.drop(["Count of CIViC Evidence Items per Category"], axis=1)

all_variant_evidence_percent_evidence_df = all_variant_evidence_df.drop(["Percentage of all CIViC Evidence Items", "Count of CIViC Evidence Items per Category"], axis=1)

In [ ]:
for_merge_all_variant_evidence_percent_of_civic_df.to_csv('../for_merge_all_variant_evidence_percent_of_civic_df.csv', index=False)

Summary Table 6: The table below shows what percentage of all evidence items in CIViC are associated with Normalized, Unable to Normalize, and Not Supported variants. This will not add up to 100% because evidence itmes may be used across multiple variants.

In [ ]:
all_variant_evidence_percent_of_civic_df = all_variant_evidence_percent_of_civic_df.set_index("Variant Category")
all_variant_evidence_percent_of_civic_df

In [ ]:
civic_summary_table_6 = all_variant_evidence_percent_of_civic_df

Summmary Table 7: The table below shows the percentage of accepted and sumbitted evidence items per category of variants. 

In [ ]:
all_variant_evidence_percent_evidence_df = all_variant_evidence_percent_evidence_df.set_index("Variant Category")
all_variant_evidence_percent_evidence_df

In [ ]:
civic_summary_table_7 = all_variant_evidence_percent_evidence_df

In [ ]:
not_supported_variant_evidence_df = pd.DataFrame(not_supported_variant_evidence_summary)

In [ ]:
not_supported_variant_evidence_df["Percent of all CIViC Evidence Items"] = not_supported_variant_evidence_df["Fraction of CIViC Evidence Items"].astype(str) + "  (" + not_supported_variant_evidence_df["Percent of all CIViC Evidence Items"] + ")"
not_supported_variant_evidence_df["Percent of Not Supported Variant Evidence Items"] = not_supported_variant_evidence_df["Fraction of Not Supported Variant Evidence Items"].astype(str) + "  (" + not_supported_variant_evidence_df["Percent of Not Supported Variant Evidence Items"] + ")"
not_supported_variant_evidence_df["Percent of Accepted Evidence Items"] = not_supported_variant_evidence_df["Fraction of Accepted Evidence Items"].astype(str) + "  (" + not_supported_variant_evidence_df["Percent of Accepted Evidence Items"] + ")"
not_supported_variant_evidence_df["Percent of Submitted Evidence Items"] = not_supported_variant_evidence_df["Fraction of Submitted Evidence Items"].astype(str) + "  (" + not_supported_variant_evidence_df["Percent of Submitted Evidence Items"] + ")"

In [ ]:
not_supported_variant_evidence_df = not_supported_variant_evidence_df.drop(["Fraction of CIViC Evidence Items", "Fraction of Not Supported Variant Evidence Items", "Fraction of Accepted Evidence Items", "Fraction of Submitted Evidence Items"], axis=1)

In [ ]:
for_merge_not_supported_variant_evidence_percent_of_civic_df = not_supported_variant_evidence_df.drop(["Percent of Not Supported Variant Evidence Items", "Percent of Accepted Evidence Items", "Percent of Submitted Evidence Items"], axis=1)

not_supported_variant_evidence_percent_of_civic_df = for_merge_not_supported_variant_evidence_percent_of_civic_df.drop(["Count of CIViC Evidence Items per Category"], axis=1)

not_supported_variant_evidence_percent_of_not_supported_df = not_supported_variant_evidence_df.drop(["Percent of all CIViC Evidence Items", "Percent of Accepted Evidence Items", "Percent of Submitted Evidence Items", "Count of CIViC Evidence Items per Category"], axis=1)

not_supported_variant_evidence_percent_evidence_df = not_supported_variant_evidence_df.drop(["Percent of Not Supported Variant Evidence Items", "Percent of all CIViC Evidence Items", "Count of CIViC Evidence Items per Category"], axis=1)

In [ ]:
for_merge_not_supported_variant_evidence_percent_of_civic_df.to_csv('../for_merge_not_supported_variant_evidence_percent_of_civic_df.csv', index=False)

Summary Table 8: The table below shows the percentage of all CIViC evidence items that are associated with a Not Supported variant sub category. This will not add up to 100% since the evidence items can be associated with multiple variants.

In [ ]:
not_supported_variant_evidence_percent_of_civic_df = not_supported_variant_evidence_percent_of_civic_df.set_index("Category")
not_supported_variant_evidence_percent_of_civic_df

In [ ]:
civic_summary_table_8 = not_supported_variant_evidence_percent_of_civic_df

Summary Table 9: The table below shows the percentage of all evidence items associated with Not Supported variants that are associated with a variant sub category. 

In [ ]:
not_supported_variant_evidence_percent_of_not_supported_df = not_supported_variant_evidence_percent_of_not_supported_df.set_index("Category")
not_supported_variant_evidence_percent_of_not_supported_df

In [ ]:
civic_summary_table_9 = not_supported_variant_evidence_percent_of_not_supported_df

Summary Table 10: The table below shows the percentage of evidence items associated with Not Supported variant sub categories that are accepted or submitted.

In [ ]:
not_supported_variant_evidence_percent_evidence_df = not_supported_variant_evidence_percent_evidence_df.set_index("Category")
not_supported_variant_evidence_percent_evidence_df

In [ ]:
civic_summary_table_10 = not_supported_variant_evidence_percent_evidence_df

## Impact

The bar graph below shows the relationship between the Not Suported variant sub category impact score and the sub category. Additionally, the colors illustrate the number of evidence items associated each sub category.

In [ ]:
fig = px.bar(
    not_supported_variant_impact_df,
    x="Category",
    y="CIVIC Total Sum Impact Score",
    hover_data=["Total Number Evidence Items", not_supported_variant_impact_df["% Accepted Evidence Items"]],
    color="Total Number Evidence Items",
    labels={"CIVIC Total Sum Impact Score": "CIVIC Total Sum Impact Score"},
    text_auto=".1f",
    color_continuous_scale="geyser"
)
fig.update_traces(width=1)
fig.show()

In [ ]:
fig.write_html("civic_ns_categories_impact_redgreen.html")

The scatterplot below shows the relationship between the Not Suported variant sub category impact score and the number of evidence items associated with variants in each sub category. Additionally, the sizes of the data point represent the number of variants in each sub category. 

In [ ]:
fig2 = px.scatter(
    data_frame=not_supported_variant_impact_df,
    x="Total Number Evidence Items",
    y="CIVIC Total Sum Impact Score",
    size="Total Number Variants",
    size_max=40,
    text="Total Number Variants",
    color="Category",
    hover_data="% Accepted Evidence Items"
)
fig2.show()

In [ ]:
fig2.write_html("civic_ns_categories_impact_scatterplot.html")

In [ ]:
fig3= px.scatter(data_frame= not_supported_variant_impact_df,
                x= 'Total Number Variants',
                y= 'Average Impact Score per Evidence Item',
                size= 'Total Number Variants',
                size_max= 40,
                text= 'Total Number Variants',
                # color_discrete_sequence= Bold,
                color= 'Category',
                hover_data= ['% Accepted Evidence Items', 'Average Impact Score per Variant'])
fig3.show()

In [ ]:
def draw_pie(dist, 
             xpos, 
             ypos, 
             size, 
             ax=None):
    if ax is None:
        fig, ax = plt.subplots(figsize=(20,8))

    # for incremental pie slices
    cumsum = np.cumsum(dist)
    cumsum = cumsum/ cumsum[-1]
    pie = [0] + cumsum.tolist()

    for r1, r2 in zip(pie[:-1], pie[1:]):
        angles = np.linspace(2 * np.pi * r1, 2 * np.pi * r2)
        x = [0] + np.cos(angles).tolist()
        y = [0] + np.sin(angles).tolist()

        xy = np.column_stack([x, y])

        ax.scatter([xpos], [ypos], marker=xy, s=size)

    return ax

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fig, ax = plt.subplots(figsize=(40,20))
for (x,y), d in not_supported_variant_impact_df.groupby(['Total Number Evidence Items','CIVIC Total Sum Impact Score']):
    dist = d['Category'].value_counts()
    draw_pie(dist, x, y, 10000*len(d), ax=ax)
plt.show()